# WiFi Setup (No Coding Required)

Use this notebook to connect the Pi to your WiFi. You do **not** need to edit any code.

**Steps**
1. Run the cell below (the one with the input boxes).
2. Enter your WiFi name (SSID).
3. Enter your WiFi password (leave blank if the network is open).
4. Click **Connect** and wait for the result message.

**What happens next**
- If the connection works, the Pi will move to your WiFi.
- If it fails or there is no internet, the Pi will bring back its hotspot so you can try again.

If you lose this page after clicking Connect, join your home WiFi and reopen the notebook.


In [1]:
# WIFI_WIDGET_UI
import importlib.util
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path

runner_path = Path("/home/pi/RaspberryPi-CAN/Wifi_Update/wifi_setup_runner.py")
if not runner_path.exists():
    raise FileNotFoundError(f"Runner not found: {runner_path}")

spec = importlib.util.spec_from_file_location("wifi_setup_runner", runner_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)
run_wifi_setup = module.run_wifi_setup

ssid_input = widgets.Text(
    description="WiFi SSID:",
    placeholder="Your WiFi name",
    layout=widgets.Layout(width="420px")
)

# Some ipywidgets versions lack Password; fallback to Text if needed
PasswordWidget = getattr(widgets, "Password", widgets.Text)
password_input = PasswordWidget(
    description="Password:",
    placeholder="Leave blank for open network",
    layout=widgets.Layout(width="420px")
)
connect_button = widgets.Button(
    description="Connect",
    button_style="success"
)
status_output = widgets.Output()

def _run_wifi_setup(_):
    ssid = ssid_input.value.strip()
    password = password_input.value
    with status_output:
        status_output.clear_output()
        if not ssid:
            print("Please enter a WiFi SSID. Leave password blank for open networks.")
            return
        run_wifi_setup(ssid, password)

connect_button.on_click(_run_wifi_setup)
display(widgets.VBox([ssid_input, password_input, connect_button, status_output]))


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


---

# 🔧 Utility Functions

Use the functions below to manage your WiFi connection manually if needed.

## Check Connection Status

Run this to see your current connection status and internet access:

In [ ]:
import subprocess

def run_cmd(cmd):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=15)
        return result.returncode == 0, result.stdout, result.stderr
    except Exception as e:
        return False, "", str(e)

print("🔍 Current Connection Status")
print("=" * 40)

# Show active connections
success, stdout, _ = run_cmd("sudo nmcli connection show --active")
if success:
    print("📡 Active connections:")
    for line in stdout.strip().split('\n')[1:]:  # Skip header
        if line.strip():
            parts = line.split()
            if len(parts) >= 4:
                name = parts[0]
                conn_type = parts[2]
                device = parts[3]
                print(f"  • {name:15} ({conn_type}) on {device}")

# Check internet
success, _, _ = run_cmd("ping -c 2 -W 5 8.8.8.8")
if success:
    print("✅ Internet: Working")
else:
    print("❌ Internet: Not working")

# Show IP
success, stdout, _ = run_cmd("ip route get 8.8.8.8 | awk '{print $7; exit}'")
if success and stdout.strip():
    print(f"📍 IP Address: {stdout.strip()}")

## Switch to Hotspot Mode

Run this to manually switch to hotspot mode (lectec-ap):

In [ ]:
import subprocess

def run_cmd(cmd):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=15)
        return result.returncode == 0, result.stdout, result.stderr
    except Exception as e:
        return False, "", str(e)

def get_hotspot_name():
    """Return the hotspot connection name"""
    return "lectec-ap"

def get_current_hostname():
    """Get current hostname (which matches the hotspot SSID)"""
    success, stdout, _ = run_cmd("hostname")
    if success and stdout.strip():
        return stdout.strip()
    return "ltpi-unknown"

print("🔄 Switching to hotspot mode...")

hotspot_name = get_hotspot_name()
current_ssid = get_current_hostname()

# Bring down WiFi connections
run_cmd("sudo nmcli connection down main-wifi 2>/dev/null")

# Start hotspot
success, _, stderr = run_cmd(f"sudo nmcli connection up {hotspot_name}")
if success:
    print("✅ Hotspot activated")
    print(f"📡 SSID: {current_ssid}")
    print("🔑 Connect to this network from your device")
else:
    print(f"❌ Failed to start hotspot: {stderr}")


## Switch to WiFi Mode

Run this to manually switch to WiFi (if main-wifi is configured):

In [ ]:
import subprocess
import time

def run_cmd(cmd):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=15)
        return result.returncode == 0, result.stdout, result.stderr
    except Exception as e:
        return False, "", str(e)

def get_hotspot_name():
    """Return the hotspot connection name"""
    return "lectec-ap"

def get_current_hostname():
    """Get current hostname (which matches the hotspot SSID)"""
    success, stdout, _ = run_cmd("hostname")
    if success and stdout.strip():
        return stdout.strip()
    return "ltpi-unknown"

print("🔄 Switching to WiFi...")

hotspot_name = get_hotspot_name()
current_ssid = get_current_hostname()

# Check if main-wifi exists
success, _, _ = run_cmd("sudo nmcli connection show main-wifi")
if not success:
    print("❌ main-wifi connection not configured. Use the setup above first.")
else:
    # Bring down hotspot
    run_cmd(f"sudo nmcli connection down {hotspot_name} 2>/dev/null")
    
    # Start WiFi
    success, _, stderr = run_cmd("sudo nmcli connection up main-wifi")
    if success:
        print("✅ WiFi activated")
        
        # Test internet
        time.sleep(3)
        success, _, _ = run_cmd("ping -c 2 -W 5 8.8.8.8")
        if success:
            print("✅ Internet access confirmed")
        else:
            print("⚠️  WiFi connected but no internet")
    else:
        print(f"❌ Failed to connect to WiFi: {stderr}")
        print("🔄 Restoring hotspot...")
        run_cmd(f"sudo nmcli connection up {hotspot_name}")
        print(f"📡 Hotspot {current_ssid} restored")
